<a href="https://colab.research.google.com/github/ordevoir/Algorithms/blob/main/Copy_of_DL_NLP_01_CharRNN_Making_Reusing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character RNN

Character RNN (или Char RNN) может прогнозировать следующий символ в предложении. Построим и обучим Char RNN, способную генерировать новый текст.

## Создание тренировочного датасета

Загрузим все работы Шекспира функцией `tf.keras.utils.get_file()`. Данные загружаются из проекта Андрея Карпаты.

In [ ]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"  # shortcut URL
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()
filepath        # путь по которому лежит загруженный файл

1115394/1115394 [==============================] - 0s 0us/step


'/root/.keras/datasets/shakespeare.txt'

In [ ]:
print(shakespeare_text[:173])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.


Воспользуемся слоем `TextVectorization` (который описан в файле Load_and_Preprocesing) для кодирования этого текста. Установим `split="character"` для получения кодирования на уровне символов, а не слов, как то задано по умолчанию. Также установим `standardize="lower"` для конвертирования текста в нижний регистр (это упростит задачу):

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",
                                                   standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]
encoded     # кодированный текст

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([21,  7, 10, ..., 22, 28, 12])>

Каждому символу поставлено в соответствие целое число, начиная с 2 (1 зарезервирована под неизвестные символы, а 0 – для отступов). Мы не нуждаемся в этих двух токенах, поэтому вычтем 2 из массива `encoded`, посчитаем число токенов и число символов в датасете:

In [ ]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)
n_tokens, dataset_size

(39, 1115394)

Теперь превратим эту длинную последовательность в датасет окон для того, чтобы использовать их при обучении sequence-to-sequence RNN. Цели (targets) будут совпадать с входами (inputs), но с единичным смещением в будущее. Например, один образец в датасете может быть полсдеовательностью ID символов, представляющих текст "to be or not to b", и соответствующий ему target – последовательность ID символов, представляющих текст "o be or not to be". Напишем функцию, которая конвертирует длинную последовательность ID символов в датасет input/target пар:

In [ ]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

Статический метод `from_tensor_slices()` превратит все элементы последовательности в отдельные образцы. Методы `from_tensor_slices()`, `batch()`, `window()`, `map()` `flat_map()` , `batch()` и `shuffle()` – описаны в Load_and_Preprocessing_Data. Разбор некоторых компонентов функции `to_dataset()` приведен ниже.

Теперь можно создать тренировочный, валидационный и тестовый наборы. 90% будет отведено на тренировочный набор, а на валидационный и тестовый – по 5%.

In [ ]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length,
                       shuffle=True)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length = length)
test_set = to_dataset(encoded[1_060_000:], length=length)

Датасет `train_set` содержит последовательность кортежей, содрежащих пару (input, ouput). Каждый input представляет собой тензор с формой (32, 100), т.е. партия из 32 образцов, а каждый образец – длиной в 100 токенов. Такую же форму имеет и output, и отличается от input лишь тем, что каждый образец смещен на 1 вперед во времени.

In [ ]:
for item in train_set:
    print(f"input:\n", item[0])
    print(f"ouput:\n", item[1])
    break

input:
 tf.Tensor(
[[ 1  9  2 ...  0 12  5]
 [ 9  5 13 ... 10  4  9]
 [ 2  6  1 ...  3 27  2]
 ...
 [ 3 22  1 ...  0 14  1]
 [20 20  4 ...  0  9  3]
 [ 2  6  1 ...  0 22  1]], shape=(32, 100), dtype=int64)
ouput:
 tf.Tensor(
[[ 9  2 11 ... 12  5 12]
 [ 5 13  7 ...  4  9 12]
 [ 6  1  7 ... 27  2 26]
 ...
 [22  1 23 ... 14  1 17]
 [20  4  8 ...  9  3  0]
 [ 6  1 15 ... 22  1  3]], shape=(32, 100), dtype=int64)


Мы установили длину окна равной 100, но можно пробовать ее менять: RNN легче и быстрее обучать на более коротких входных последовательностях, но при этом она не сможет изучить какой-либо паттерн длиннее `length`, поэтому не следует делать ее слишком маленькой.

### Разбор функции `to_dataset()`

In [ ]:
seq = tf.range(7)      # создаем последовательность
print(seq)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)


Создаем из последовательности объект `Dataset`, в котором образцы представлены в виде тензоров (в данном случае нулевого ранга).

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(seq)
print(type(ds))
for item in ds:
    print(f"{item}", end=' ')
for item in ds:
    print(f"\n{type(item)} \nтензор нулевого ранга:")
    print(item)
    break

<class 'tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset'>
0 1 2 3 4 5 6 
<class 'tensorflow.python.framework.ops.EagerTensor'> 
тензор нулевого ранга:
tf.Tensor(0, shape=(), dtype=int32)


Получим окна (объект класса `_WindowDataset`) со сдвигом 1:

In [ ]:
length = 3
ds_windows = ds.window(size=length+1, shift=1, drop_remainder=True)
print(f"{type(ds_windows) = }")
for window in ds_windows:
    for e in window:
        print(f"{e}", end=' ')
    print(f" window, {type(e)}")

type(ds_windows) = <class 'tensorflow.python.data.ops.window_op._WindowDataset'>
0 1 2 3  window, <class 'tensorflow.python.framework.ops.EagerTensor'>
1 2 3 4  window, <class 'tensorflow.python.framework.ops.EagerTensor'>
2 3 4 5  window, <class 'tensorflow.python.framework.ops.EagerTensor'>
3 4 5 6  window, <class 'tensorflow.python.framework.ops.EagerTensor'>


Сейчас образцы в `ds` представляют собой объекты класса `_VariantDataset`:

In [ ]:
for item in ds_windows:
    print(type(item))

<class 'tensorflow.python.data.ops.dataset_ops._VariantDataset'>
<class 'tensorflow.python.data.ops.dataset_ops._VariantDataset'>
<class 'tensorflow.python.data.ops.dataset_ops._VariantDataset'>
<class 'tensorflow.python.data.ops.dataset_ops._VariantDataset'>


При вызове метода `flat_map()`, все отдельные объекты `_VariantDataset` будут выпрямлены в один объект `_VariantDataset` и переданы в преобразующую [`lambda`] функцию. Функция разобъет эту общую последовательность на последовательности длины `length+1` методом `batch()`. Фактически мы конвертируем объекты класса `_VariantDataset` в тензоры.

In [ ]:
ds_flatten = ds_windows.flat_map(lambda v: v.batch(length+1))

for item in ds_flatten:
    print(item)

tf.Tensor([0 1 2 3], shape=(4,), dtype=int32)
tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)
tf.Tensor([2 3 4 5], shape=(4,), dtype=int32)
tf.Tensor([3 4 5 6], shape=(4,), dtype=int32)


Разбиваем последовательность тензоров на партии:

In [ ]:
batch_size = 2
ds_batched = ds_flatten.batch(batch_size)
for item in ds_batched:
    print(item)

tf.Tensor(
[[0 1 2 3]
 [1 2 3 4]], shape=(2, 4), dtype=int32)
tf.Tensor(
[[2 3 4 5]
 [3 4 5 6]], shape=(2, 4), dtype=int32)


Осталось получить inputs и outputs:

In [ ]:
result = ds_batched.map(lambda window: (window[:, :-1], window[:, 1:]))
for item in result:
    print(item)

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[0, 1, 2],
       [1, 2, 3]])>, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [2, 3, 4]])>)
(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 4],
       [3, 4, 5]])>, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[3, 4, 5],
       [4, 5, 6]])>)


Итоговый датасет представляет собой последовательность кортежей, состоящих из пар тензоров (inputs, outputs).

![](https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/images/nn/shuffled_windows.png)

## Построение и обучение модели

Построим модель с GRU слоем, составленным из 128 нейронов и обучим ее:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam", metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "shakespeare_model", monitor="val_accuracy",
    save_best_only=True)
history = model.fit(train_set, validation_data=valid_set,
                    epochs=10, callbacks=[model_ckpt])

Epoch 1/10


KeyboardInterrupt: 

Здесь используется слой `Embedding`, описанный в Load_and_Preprocessing_Data. Этот слой ставит в соответствие IDs символов точки в 16-мерном пространстве. На вход слой будет получать 2D тензоры с формой [*batch size, window length*], а возврщащать слой будет 3D тензор с формой [*batch size, window length, embedding size*] (embedding size в данном случае задан как `output_dim=16`).

Слой `Dense` должен состоять из 39 нейронов (`n_tokens`), так как всего имеется 39 различных символов в тексте, и мы хотим получать вероятности для каждого возможного символа (на каждом временнóм шаге). В сумме все эти вероятности должны давать 1 на каждом временнóм шаге, так что применяется функция активации softmax.

Данная модель не производит предобработку текста, поэтому имеет смысл обернуть ее в финальную модель, которая будет содержать слой `tf.keras.layers.TextVectorization` в качестве первого слоя. Также добавим слой `tf.keras.layers.Lambda` для вычитания 2 из IDs символов, так как мы не будтем использовать отступы и неизестные токены:

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

### Загрузка обученной модели

Обучение на GPU в Colab может занять несколько часов. Можно скачать предобученную Geron'ом модель.

In [ ]:
from pathlib import Path

url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = tf.keras.utils.get_file("shakespeare_model.tgz", url, extract=True)
model_path = Path(path).with_name("shakespeare_model")
print(model_path)

C:\Users\wernadsky\.keras\datasets\shakespeare_model


Используем загруженную модель вместо построенной выше:

In [ ]:
shakespeare_model = tf.keras.models.load_model(model_path)

Проверим на известном примере:

In [ ]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 0s 418ms/step


'e'

## Генерация текста

Для того, чтобы сгенерировать текст, используя модель char-RNN, мы должны скормить ей некоторый текст, дать модели спрогнозировать наибольее вероятную следующую букву, и добавить это в конец текста, задем дать дать этот расширенный текст модели для угадывания следующей буквы и тд. Это называется **greedy decoding**. Но на практике это часто приводит к тому, что одно и то же слово повторяется снова и снова. Вместо этого, мы можем выбрать следующий символ случайно, с соответствии с распределением вероятностей, данным в прогнозе. Для этого можно использовать функцию `tf.random.categorical()`, которая будет генерировать более разнообразный и интересный текст.

Функция `tf.random.categorical()` принимает логарифмированные вероятности классов, и выбирает индекс класса. В аргументе `num_samples` можно задать количество генерируемых обазцов.

In [ ]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.categorical(log_probas, num_samples=8)  # draw 8 samples

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[1, 0, 1, 1, 1, 0, 2, 1]], dtype=int64)>

Для того, чтобы иметь больше контроля над разнообразием генерируемого текста, мы можем разделить логарифмы на число, называемое **температурой**. Температура, близкая к нулю способствует "контрастированию" вероятностей символов, в то время как высокая температура уравнивает вероятности для всех символов. Низкая температура обычно предпочтительна при генерации довольно ригидного и точного текста, такого как математическое уравнение, в то время как высокая температура предпочтительней при генерации более разнообразного и креативного текста.

Функция `next_char()` будет помагать выбрать следующий символ для добавления во входящий текст:

In [ ]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits,
                                    num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

Далее мы можем написать другую вспомогательную функцию `extend_text()`, которая будет вызывать в цикле функцию `next_char()` для получения следующего символа и добавления его в заданный текст:

In [ ]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

Произведем генерацию при различных температурах (0.01, 1, 100):

In [ ]:
from IPython.display import clear_output
generated = extend_text("To be or not to b ", temperature=0.01)
clear_output()
print(generated)

To be or not to b on the strange daughter
to the death and the death


In [ ]:
tf.random.set_seed(42)
generated = extend_text("To be or not to b", temperature=.5)
clear_output()
print(generated)

To be or not to be but a commody to her provost.

claudio:
why do you speak to angelo.

petruchio:
a playalliats, thou hast the heavens so that course.

lucio:
i am a pains he hath made the very noble thing,
so i would not see the issue of the news with his life
to death is not you a thousand haste to sent to heard your master's
head to me the head should be the sacred a state,
and would i know him and here, i know this sheewer,
and i think it is a streak with the prison,
and make me too much soundly be to execu


In [ ]:
generated = extend_text("To be or not to be", temperature=100)
clear_output()
print(generated)

To be or not to berhpmflkbn'ojsgeuwt?o!r.s?iszc
rgabtfd$
peruvdok.t.


Для генерации более убедительного текста, обычной практикой является отбор только из $k$ наиболее вероятных символов, или только из небольшого набора символов, чьи вероятности превышают некоторый порог (это называется *nucleus sampling*).

Алтернативный вариант – использовать *beam search* (см. ниже), или использование большего чисало GRU слоев и больше нейронов на слой, более длительное обучение, и добавление регуларизации при необходимости.

Следует также заметить, что модель не способна уловить паттерны длиннее `lenght`, который составляет 100 символов. Можно попробовать сделать ширину окна больше, но это также усложнит обучение, и даже LSTM и GRU cells не смогут обрабатывать очень длинные последовательности. Альтернативным вариантом является использование stateful RNN.

## Stateful RNN

До сих пор мы использовали только **stateless RNNs**: на каждом экземпляре (полсдовательность из 100 шагов) модель стартует с hidden state, заполненного нулями, и это состояние обновляется на каждом временнóм шаге, и после последнего шага состояние выбрасывается за ненадобностью. Поэтому, на текущем временном шаге влияние оказывают лишь предыдущие значения данной последовательности, в то время как предыдущая посделовательность влияния не оказывает, так как hidden state не хранит о них информацию. Таким образом, stateles RNN обрабатывает последовательности независимо.

**Stateful RNNs** характерен тем, что hidden state сохраняет память между последовательностями, улавливая долгосрочные зависимости. Поэтому, stateful RNNs используются, когда порядок и непререрывность последовательностей имеет существенное значение. В stateful RNN скрытое состояние RNN после обработки одной последовательности используется как начальное состояние для следующей последовательности.

Будем предполагать, что в начале прохождения $i$-ой последовательности текущей партии, hidden state инициализируется финальным значением hidden state после прохождения на $i$-ой последовательности *предыдущей* партии. Поэтому, при формировании партий, каждая $i$-ая последовательность партии длжна быть продолжением $i$-ой последовательности предыдущей партии.

### Подготовка данных

Для того, чтобы чтобы использовать stateful RNN, необходимо иначе подготовить данные, чтобы последовательности шли друг за другом последовательно, без перекрытий (выше мы использовали перекрывающиеся последовательности, и еще перемешивали их). При создании объекта `tf.data.Dataset` мы долджны использовать `shift=lenght` (вместо `shift=1`), затем вызвыать метод `window()`. И, конечно, мы не долждны вызывать метод `shuffle()`.

Формирование партий посложней, когда данные препарируются для stateful RNN, чем для stateless RNN. Предположим, мы разбили исходный текст так, что имется последовательность окон, каждое из которых является продолжением предыдущего окна. Так, если просто вызвать `batch(32)`, то 32 окна, следующих друг за другом, будут помещены в одну партию, а следующие 32 окна – в следующую партию. Это нам не подходит, так как каждая $i$-ая последовательность партии длжна быть продолжением $i$-ой последовательности предыдущей партии. Простейшее решение этой проблемы – использовать партии размером 1:

In [ ]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

length = 100
stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000], length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

In [ ]:
for item in stateful_train_set:
    print(f"input:\n", item[0])
    print(f"ouput:\n", item[1])
    break

input:
 tf.Tensor(
[[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1
   0 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1
   4  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24
  17  0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23
  10 15  3 13]], shape=(1, 100), dtype=int64)
ouput:
 tf.Tensor(
[[ 5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0
  22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4
   8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24 17
   0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23 10
  15  3 13  0]], shape=(1, 100), dtype=int64)


Создать партии в общем-то можно. К примеру, можно нарезать весь текст на 32 фрагмента равной длины, создать один датасет отбирая из каждого фрагмента последовательность по очереди. Получим 32 последовательности из каждого фрагмента, затем еще 32 последовательности и так по кругу, пока фрагменты не исчерпаются.

![](https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/images/nn/prepare_for_stateful_rnn.png)

In [ ]:
import numpy as np

def to_non_overlapping_windows(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    return ds.flat_map(lambda window: window.batch(length + 1))

def to_batched_dataset_for_stateful_rnn(sequence, length, batch_size=32):
    parts = np.array_split(sequence, batch_size)
    datasets = tuple(to_non_overlapping_windows(part, length) for part in parts)
    ds = tf.data.Dataset.zip(datasets)
    ds = ds.map(lambda *windows: tf.stack(windows))
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

list(to_batched_dataset_for_stateful_rnn(tf.range(20), length=3, batch_size=2))

[(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 0,  1,  2],
         [10, 11, 12]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 1,  2,  3],
         [11, 12, 13]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 3,  4,  5],
         [13, 14, 15]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 4,  5,  6],
         [14, 15, 16]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 6,  7,  8],
         [16, 17, 18]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 7,  8,  9],
         [17, 18, 19]])>)]

>Ниже приведен подробный разбор функций

In [ ]:
length = 100
batch_stateful_train_set = to_batched_dataset_for_stateful_rnn(encoded[:1_000_000], length)
batch_stateful_valid_set = to_batched_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000], length)
batch_stateful_test_set = to_batched_dataset_for_stateful_rnn(encoded[1_060_000:], length)

### Построение и обучение модели

Создадим модель stateful RNN. Для этого в слое `GRU` необходимо задать аргумент `stateful=True`, и так как это stateful RNN, сеть должна знать размер партии (так как он должен будет хранить состояние для каждой входной последовательности в партии). Таким образом, мы должны задать аргумент `batch_input_shape` в первом слое. Заметим, что мы можем оставить вторую размерность неопределенной (`None`), так как в общем случае входные последовательности для RNN могут иметь любую длину:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                              batch_input_shape=[32, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

В конце каждой эпохи мы должны сбрасывать состояние, прежде чем мы будем возвращаться к началу текста. Для этого мы можем написать пользовательский callback, который в начале каждой эпохи будет сбрасывать состояние, вызывая у модели метод `reset_states()`:

In [ ]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam", metrics=["accuracy"])
history = model.fit(batch_stateful_test_set,
                    validation_data=batch_stateful_valid_set,
                    epochs=10, callbacks=[ResetStatesCallback()])

Epoch 1/10
17/17 [==============================] - 4s 104ms/step - loss: 3.5668 - accuracy: 0.1339 - val_loss: 3.1944 - val_accuracy: 0.1521
Epoch 2/10
17/17 [==============================] - 1s 66ms/step - loss: 3.1282 - accuracy: 0.1461 - val_loss: 3.0831 - val_accuracy: 0.1521
Epoch 3/10
17/17 [==============================] - 1s 67ms/step - loss: 3.0793 - accuracy: 0.1461 - val_loss: 3.0663 - val_accuracy: 0.1521
Epoch 4/10
17/17 [==============================] - 1s 66ms/step - loss: 3.0698 - accuracy: 0.1461 - val_loss: 3.0587 - val_accuracy: 0.1521
Epoch 5/10
17/17 [==============================] - 1s 71ms/step - loss: 3.0617 - accuracy: 0.1461 - val_loss: 3.0485 - val_accuracy: 0.1521
Epoch 6/10
17/17 [==============================] - 1s 75ms/step - loss: 3.0480 - accuracy: 0.1461 - val_loss: 3.0299 - val_accuracy: 0.1521
Epoch 7/10
17/17 [==============================] - 1s 76ms/step - loss: 3.0224 - accuracy: 0.1462 - val_loss: 2.9953 - val_accuracy: 0.1521
Epoch 8/10
1

>После того, как модель обучена, прогнозы можно будет получить только для партий того же размера, что использовалось в обучении. Для того, чтобы снять такое ограничение, можно создать такую же stateless сеть, и скопировать в нее параметры stateful модели.

Интересно, что хотя модель char-RNN просто обучена предсказывать следующий символ, эта, казалось бы, простая задача на самом деле требует от нее также изучения некоторых паттернов более высокого уровня. Например, чтобы найти следующий символ после «Отличный фильм, я действительно», полезно понимать, что предложение положительное, поэтому далее, скорее всего, будет буква «л» (что означает «любимый»), а не «н». (для «ненавижу»). Фактически, в статье Alec Radford и других исследователей OpenAI, опубликованной в 2017 году, описывается, как авторы обучили большую charRNN-подобную модель на большом наборе данных и обнаружили, что один из нейронов действует как отличный классификатор анализа настроения (*sentiment neuron*): хотя модель была обучена без каких либо меток настроения, нейрон настроения достиг самых высоких результатов в тестах анализа настроений. Это мотивировало к предварительном обучению NLP без учителя.

### Разбор функций

Функция `to_non_overlapping_windows()` похожа на `to_dataset_for_stateful_rnn()` но без хвоста (batching). Она возвращает датасет из непересекающихся окон, каждое из которых является продолжением следующего. A batching осуществляется функцией `to_batched_dataset_for_stateful_rnn()`. Она берет исходную последовательность, и нарезает ее на 32 (`batch_size`) фрагмента, используя функцию `np.array_split()`. К каждой из фрагментов применяется функция `to_non_overlapping_windows()` и результаты (окна) собираются в кортеж `datasets`:

In [ ]:
import numpy as np
import tensorflow as tf

batch_size = 4
length = 5
sequence = np.arange(60)
parts = np.array_split(sequence, batch_size)
datasets = tuple(to_non_overlapping_windows(part, length) for part in parts)
datasets

(<_FlatMapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>,
 <_FlatMapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>,
 <_FlatMapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>,
 <_FlatMapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>)

Статический метод `zip()` проходится параллельно по элементам кортежа, и на каждой итерации отбирает по одному окну, собирая каждый раз по 32 окна в новый кортеж. Метод возвращает объект класса `_ZipDataset`, содержащий кортежи длины `batch_size`:

In [ ]:
ds = tf.data.Dataset.zip(datasets)
print(f"{type(ds) = }")
for e in ds:
    print(f"{len(e)  = }")
    print(f"{type(e) = }")
    break

type(ds) = <class 'tensorflow.python.data.ops.zip_op._ZipDataset'>
len(e)  = 4
type(e) = <class 'tuple'>


Посмотрим, что за кортежи:

In [ ]:
for e in ds:
    for item in e:
        print(item)
    print()

tf.Tensor([0 1 2 3 4 5], shape=(6,), dtype=int32)
tf.Tensor([15 16 17 18 19 20], shape=(6,), dtype=int32)
tf.Tensor([30 31 32 33 34 35], shape=(6,), dtype=int32)
tf.Tensor([45 46 47 48 49 50], shape=(6,), dtype=int32)

tf.Tensor([ 5  6  7  8  9 10], shape=(6,), dtype=int32)
tf.Tensor([20 21 22 23 24 25], shape=(6,), dtype=int32)
tf.Tensor([35 36 37 38 39 40], shape=(6,), dtype=int32)
tf.Tensor([50 51 52 53 54 55], shape=(6,), dtype=int32)



Здесь мы видим, что $i$-ый элемент кортежа представляет собой последовательность (`Tensor`), которая является продолжением $i$-го элемента предыдущего кортежа.

>Здесь, правда, есть наложение: третий элемент второго кортежа начинается с 35, а третий элемент первого кортежа заканчивается также на 35. Но заметим, что длины этих последовательностей равны 6 (`length+1`). При формировании inputs будут использованы первые 5 элементов последовательности, а при формированиии outputs – последние 5. Поэтому в результирующих партиях наложения не будет.

Фактически, партии уже распределены, остается лишь превратить кортежи в тензоры и сформировать inputs и outputs.

Воспользуемся методом `map()`, для того, чтобы объединить элементы (тензоры ранга 2) каждого кортежа в тензор ранга 2. Датасет `ds` состоит из двух кортежей, каждый из которых содержит по 4 тензора. Поэтому, при вызове метода `map()` в обрабатывающую [`lambda`] функцию будут передаваться тензоры из кортежа, как отдельные аргументы. Так что необходимо запаковывать их в кортеж (`*windows`):

In [ ]:
ds = ds.map(lambda *windows: tf.stack(windows))
print(f"{type(ds) = }")
for item in ds:
    print(item)

type(ds) = <class 'tensorflow.python.data.ops.map_op._MapDataset'>
tf.Tensor(
[[ 0  1  2  3  4  5]
 [15 16 17 18 19 20]
 [30 31 32 33 34 35]
 [45 46 47 48 49 50]], shape=(4, 6), dtype=int32)
tf.Tensor(
[[ 5  6  7  8  9 10]
 [20 21 22 23 24 25]
 [35 36 37 38 39 40]
 [50 51 52 53 54 55]], shape=(4, 6), dtype=int32)


На данном этапе датасет `ds` представляет собой объект класса `_MapDataset`, в котором содержится 2 тензора ранга 2. Количество тензоров в общем случае зависит от исходного объема данных и соответствует итоговому количеству партий.

Теперь сформируем inputs и outputs, снова воспользовавшись методом `map()`. На этот раз элементами датасета являются тензоры, а не кортежи, поэтому в обрабатывающую функцию на каждой итерации будет передаваться один аргумент (`window`). В целом, тот этап идентичен тому, что было в `to_dataset()`:

In [ ]:
result = ds.map(lambda window: (window[:, :-1], window[:, 1:]))
for i, batch in enumerate(result):
    print(f"batch {i}:")
    for j, item in enumerate(batch):
        print(f" item {j}: \n{item}")

batch 0:
 item 0: 
[[ 0  1  2  3  4]
 [15 16 17 18 19]
 [30 31 32 33 34]
 [45 46 47 48 49]]
 item 1: 
[[ 1  2  3  4  5]
 [16 17 18 19 20]
 [31 32 33 34 35]
 [46 47 48 49 50]]
batch 1:
 item 0: 
[[ 5  6  7  8  9]
 [20 21 22 23 24]
 [35 36 37 38 39]
 [50 51 52 53 54]]
 item 1: 
[[ 6  7  8  9 10]
 [21 22 23 24 25]
 [36 37 38 39 40]
 [51 52 53 54 55]]


# Sentiment Analysis

Произведем бинарную классификацию настроения обзоров фильмов из [IMDb](https://www.imdb.com/). Обзоры будут делиться на два класса negative (0) positive (1).

## Загрузка данных IMDb

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews", as_supervised=True,
    split=["train[:90%]", "train[90%:]", "test"]
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

Keras также включает в себя функцию `tf.keras.datasets.imdb.load_data()` для загрузки датасета IMDb. При этом обзоры уже предобработаны и представлены как последовательности IDs слов.

In [ ]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8"))
    print("Label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development

## Tokenization

Для того, чтобы работать с текстом, необходимо производить препроцессинг. Разбиение текста на отдельные единицы (токены), называется **токенизацией**. В общем-то можно назвать и сегментацией. Для char-RNN мы разбивали текст на отдельные символы. Другой вариант – разбивать на слова, также используя слой `tf.keras.layers.TextVectorization`. При разбиении текста на отдельные слова, используется пробел, для детектирования границ слов. Однако, следует иметь в виду, что это не очень хорошо работает для некоторых языков: в китайском не используются пробелы между словами, а в немецком часто несколько слов сливают в одно слово без пробелов. Даже в английском пробелы не всегда являются хорошим выбором при токенизации текста: например, "San Francisco" или "#ILoveDeepLearning".

**Open Vocabulary Problem** (проблема открытого словаря)  связана со способностью модели работать со словами или фразами, которые не содержатся в предопределенном словаре. Проблема возникает в силу постоянного развития языка и создания новых слов, а так же в связи с тем, что различные области могут обладать специфическими терминами или выражениями, которые не вписываются в другие контексты.

К счастью, есть решения этих проблем. В [статье](https://arxiv.org/abs/1508.07909) Rico Sennrich 2016 года было исследовано несколько методов токенизации и детокенизации текста на уровне подслов (*subword level*). В этом подходе, если даже модель сталкивается с незнакомым словом, она все еще может догадаться о его значении. Например, даже если модель никогда не сталкивалась со словом "smartest" в процессе обучения, но при этом изучала слово "smart" и также изучила, что суффикс "est" означает "the most", она сможет сделать вывод о значении слова "smartest". Одна из таких техник – byte pair encoding (BPE). BPE разделяет весь тренировочный набор на отдельные символы (включая пробелы), затем производит слияние наиболее часто встречающихся пар символов. Далее производится слияние уже объединенных смежных элементов и так далее, пока словарь не достигнет желаемых размеров.

В [статье]() Taku Kudo 2018 года subword tokenization было улучшено. В статье предлагается новая техника регуляризации, названная **subword regularization**, которая улучшает верность и робастность путем внесения некоторой стохастичности в токенизацию в процессе обучения: к примеру, "New England" может быть токенизировано как "New" + "England" или "New" + "Eng" + "land", или просто как "New England" (всего один токен). Т.е. токенизация становится не однозначной. То обстоятельство, что в ходе обучения случайным образом выбираются различные сегменты для токенизации, увеличивает лингвистическую вариативность модели.

Библиотека [TensorFlow Text](https://www.tensorflow.org/text) ([GitHub](https://github.com/tensorflow/text), [PyPi](https://pypi.org/project/tensorflow-text/)) также реализует различные стратегии токенизации, включая WordPieces (вариант BPE). В [Tokenizers library by Hugging Face](https://huggingface.co/docs/tokenizers/index) реализован широкий спектр предельно быстрых токенизаторов.

Для токенизации в задачи классификации обзоров IMDb достаточно будет использовать пробелы для границ токенов. Так что создадим слой TextVectorization и адаптируем его к обучающему набору. Ограничим размер словаря 1000 токенами, включая 998 самых частых слов плюс токен отступа (padding token) и токен неизвестных слов, так как очень маловероятно, что низкочастотные слова будут важны для данной задачи. Ограничение размера словаря снизит число параметров, необходимых модели для обучения::

In [ ]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

text_vec_layer("This is great!")

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([11,  7, 86], dtype=int64)>

## Создание и обучение модели

In [ ]:
embed_size = 128

model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 227s 319ms/step - loss: 0.6935 - accuracy: 0.5040 - val_loss: 0.6930 - val_accuracy: 0.5024
Epoch 2/2
704/704 [==============================] - 233s 331ms/step - loss: 0.6930 - accuracy: 0.4986 - val_loss: 0.6926 - val_accuracy: 0.5036


Слой `TextVectorization` переводит слова в числа (IDs), а слой `Embedding` переводит IDs слов в embeddings. Embedding Matrix должна иметь одну строку на каждый токен в словаре (`vocab_size`) и число колонок, равное размерности (`embed_size`).

Как видно из результатов, сеть обучилась крайне плохо: accuracy едва ли отличается от 0.5. С чем это связано? Обзоры фильмов имеют разную длину, и когда слой `TextVectorization` конвертирует их в последовательности IDs токенов, он дополняет отступами короткие рецензии, используя токен отступа (ID 0) для того, чтобы сделать их длину такой же, как и самая длинная последовательность в партии. В результате, большинство последовательностей заканчиваются с большим количеством токенов отступа (это могут быть десятки и даже сотни). И хотя мы используем слой `GRU`, который гораздо лучше, чем слой `SimpleRNN`, его краткосрочная память все еще не так велика. Так что когда он проходит через множество токенов отступа, он забывает, о чем вообще была эта рецензия. Одно из решений – скармливать модели партии, в которых все последовательности имеют равные длины (что также поднимет скорость обучения). Другое решение – игнорирование токенов отступа сетью. Это может быть сделано при помощи маскировок.

## Masking

Настроить модель Keras с тем, чтобы она игнорировала токены паддингов, нужно задать `mask_zero=True` при создании слоя `Embedding`. Это значит, что токен паддинга (ID 0) будет проигнорирован всеми последющими слоями. Попробуем заново обучить такую модель:

In [ ]:
embed_size = 128

model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2



704/704 [==============================] - 267s 371ms/step - loss: 0.4786 - accuracy: 0.7676 - val_loss: 0.3876 - val_accuracy: 0.8416
Epoch 2/2
704/704 [==============================] - 256s 364ms/step - loss: 0.3351 - accuracy: 0.8604 - val_loss: 0.3115 - val_accuracy: 0.8700


Данный способ работает следующим образом: слой `Embedding` создает **маскирующий тензор** (*mask tensor*) используя функцию `tf.math.not_equal(inputs, 0)`. Эта функция возвращает булевый тензор той же формы, что и `inputs`, в котором все элменты, соответствующие нулю в `inputs`, равны `False`, а все остальные элементы равны `True`. Этот маскирующий тензор автоматически распространяется по модели к следующим слоям, и если их метод `call()` имеет аргумент `mask`, то он автоматически получает этот тензор. Это позволяет слою игнорировать соответствующие временные шаги. Каждый слой может использовать тензор по-разному, но в общем они просто игнорируют замаскированные временные шаги. Например, когда рекуррентный слой сталкивается с замаскированным временным шагом, он просто копирует output из предыдущего временнóго шага.

Если атрибут `supports_making` слоя имеет значние `True`, то маска автоматически распространяется к следующему слою. Таким образом, маскирующий тензор распространяется пока слои имеют `supports_making=True`. Например, атрибут `supports_masking` рекуррентного слоя равен `True`, когда `return_sequences=True`, и равен `False`, когда `return_sequences=False`, так как в этом случае уже нет необходимости в масировке.

В нашем случае слой `GRU` получит маску автоматически и будет ее использовать, но она не будет распространять ее дальше, так как `return_sequences=False`.

>Некоторым слоям необходимо обновить маску перед ее распространением на следующий слой: они делают это путем реализации метода `compute_mask()`, который принимает два аргумента: входные данные и предыдущую маску. Затем он вычисляет обновленную маску и возвращает ее. Метод `compute_mask()` по умолчанию просто возвращает предыдущую маску без изменений.

Также можно реализовать пользовательские слои, которые будут поддерживать маски.

Слои `LSTM` и `GRU` имеют оптимизированную реализацию для графических процессоров на основе библиотеки cuDNN от Nvidia. Однако эта реализация поддерживает маскирование только в том случае, если все токены отступов находятся в конце последовательностей. Также требуется использовать значения по умолчанию для нескольких гиперпараметров: `activation`, `recurrent_activation`, `recurrent_dropout`, `unroll`, `use_bias` и `reset_after`. В протьивном случае эти слои откатятся к (гораздо более медленной) дефолтной реализации для GPU.

Если модель не стартует слоем `Embedding`, то можно использвать слой `tf.keras.Masking`.

Использование маскирующих слоев и автоматического распространения маски лучше всего подходит для простых моделей. Это не всегда будет работать для более сложных моделей, например, когда нужно смешать слои `Conv1D` с рекуррентными слоями. В таких случаях нужно будет явно вычислить маску и передать ее соответствующим слоям, используя либо Functional API, либо Subclassing API. Например, следующая модель эквивалентна предыдущей модели, за исключением того, что она построена с использованием Functional API и добавляет маскировку вручную. Также добавим немного dropout, поскольку предыдущая модель была немного переобучена:

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on the CPU
inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
token_ids = text_vec_layer(inputs)
mask = tf.math.not_equal(token_ids, 0)
Z = tf.keras.layers.Embedding(vocab_size, embed_size)(token_ids)
Z = tf.keras.layers.GRU(128, dropout=0.2)(Z, mask=mask)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(Z)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

**Warnign**: запуск следующей ячейки займет около 30 минут, если не используется GPU.

In [ ]:
# extra code – compiles and trains the model, as usual
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Еще один способ маскировки – скармливать модели ragged tensors. На практике, все, что нужно сделать, это установить `ragged=True` при созданиии слоя `TextVectorization`, так что входные последовательности представлены как ragged tensors:

In [ ]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size, ragged=True)
text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels: reviews))
text_vec_layer_ragged(["Great movie!", "This is DiCaprio's best role."])

<tf.RaggedTensor [[86, 18], [11, 7, 1, 116, 217]]>

In [ ]:
text_vec_layer(["Great movie!", "This is DiCaprio's best role."])

<tf.Tensor: shape=(2, 5), dtype=int64, numpy=
array([[ 86,  18,   0,   0,   0],
       [ 11,   7,   1, 116, 217]], dtype=int64)>

In [ ]:
text_vec_layer_ragged(["Great movie!", "This is DiCaprio's best role."])

<tf.RaggedTensor [[86, 18], [11, 7, 1, 116, 217]]>

**Warnign**: запуск следующей ячейки займет около 30 минут, если не используется GPU.

In [ ]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer_ragged,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Рекуррентные слои Keras имеют встроенную поддержку для ragged tensors, так что нет необходимости передавать `mask_zero=True` или вручную задавать явноя маски, достаточно просто использовать слой `TextVectorization` в модели. Однако, так как поддержка ragged tensors в Keras была добавлена недавно, могут возникать ньюансы. В частности, на момент конспекта еще не было возможности использовать ragged tensors как targets при запуске на GPU (но верятно это в скором времени будет исправлено).

>Если использовать `tf.keras.callbacks.TensorBoard()`, то можно визуализировать embeddings в TensorBoard в ходе обучения, и видеть, как некоторые слова постепенно образуют кластеры.

## Reusing Pretrained Embeddings and Language Models

Вместо обучения embeddings слов, мы можем просто загрузить и использовать предобученные embeddings, такие как гугловский [Word2vec embeddings](https://www.tensorflow.org/text/tutorials/word2vec), Стенфордский [GloVe embeddings](https://nlp.stanford.edu/projects/glove/), или [FastText embeddings](https://fasttext.cc/) от Facebook.

Использование предобученных embeddings слов было популярно в течение нескольких лет, однако этот подход имеет свои ограничения. На практике слова имеют единственное представление, не учитывающее контекст (в том числе и омонимы). К примеру, слово "right" будет кодироваться одинаково как в случае "left and right" так и для "right and wrong", хотя слово имеет совершенное разные значения в этих двух контекстах. Это ограничение было устранено в [статье](https://arxiv.org/abs/1802.05365v2) Matthew Peters, в которой были введены Embeddings from Language Models (ELMo), которые представляют собой контекстуализированные embeddings слов, извлеченные из внутренних состояний глубокой двунаправленной (*bidirectional*) языковой модели. Вместо использования предобученных embeddings слов в модели, используется часть предобученной языковой модели.

Примерно в то же время в [статье](https://arxiv.org/abs/1801.06146) Jeremy Howard и Sebastian Ruder было продемонстрирована эффективность предварительного обучения без учителя для задач NLP: авторы обучили языковую модель LSTM на огромных корпусах текстов используя self-supervised learning (генерируя метки автоматически из данных), затем они надстраививали (fine-tune) модель под различные задачи (Universal Language Model Fine-Tuning – ULMFiT). Их модель превзошла современные модели в шести задачах классификации, причем, с болшим отрывом (сократив ошибку на 18-24% в большинстве случаев). Более того, авторы показали, что предобученная модель надстроенная всего на 100 размеченных образцах могла достичь тех же результатов, что и аналогичная модель, обученная с нуля на 10000 образцов. Прежде, использование предобученных моделей было нормой только в компьютерном зрении. Данная статья обозначила начало новой эры NLP: сегодня переиспользование предобученных языковых моделей является нормой.

В качестве примера построим классификатор на базе Universal Sentence Encoder – модели, введеной исследователями Google в [статье](https://arxiv.org/abs/1803.11175). Эта модель основан на архитектуре трансформеров. Модель доступна на TensorFlow Hub.

>Эта модель довольно велика – около 1 GB, так что загрузка может занять некоторое время. По умолчанию, модули TensorFlow Hub сохраняются во временной директории и загрузка повторятся каждый раз, когда запускается программа. Для того, чтобы этого избежать, необходимо задать переменной окружения `TFHUB CACHE DIR` директорию: модули будут тогда сохраняться в этой директории и загружаться лишь один раз.

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = 'tfhub_cache'
tf.random.set_seed(42)
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, validation_data=valid_set, epochs=10)

Заметим, что последняя часть URL-адреса модуля TensorFlow Hub указывает, что нам нужна версия модели 4. Такое управление версиями гарантирует, что если новая версия модуля будет выпущена на TF Hub, это не сломает нашу модель. Если ввести этот URL-адрес в веб-браузере,  то откроется документация этого модуля.

Установка аргумента `trainable=True` при создании слоя `hub.KerasLayer` позволяет производить надстройку (fine-tuning) Universal Sentence Encoder в процессе обучения: некоторые его веса будут подкручены обратным распространением ошибки. Не все модули из TF Hub поддерживают надстройку, так что следует сначала убедиться в документации.\

После обучения этп модель должна достичь accuracy выше 90% на валидационных данных. Это довольно хороший результат: если попробовать произвести классификацию самостоятельно, то результат будет не намного выше, так как многие рецензии содержат как положительные, так и отрицательные комментарии. Классификация таких двусмысленных рецензий схоже с бросанием монеты.